!pip install qdrant-client requests

In [2]:
import json
import requests
from qdrant_client import QdrantClient, models

# =========================================
# 1️⃣ 설정
# =========================================
OLLAMA_URL = "http://localhost:11434/api/embeddings"
QDRANT_URL = "http://localhost:6333"
COLLECTION_NAME = "policy_notices"
VECTOR_SIZE = 1024  # mxbai-embed-large 차원 수

# =========================================
# 2️⃣ Qdrant 클라이언트 연결
# =========================================
client = QdrantClient(url=QDRANT_URL)

# 컬렉션 새로 생성 (기존 존재 시 덮어쓰기)
client.recreate_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=models.VectorParams(size=VECTOR_SIZE, distance=models.Distance.COSINE),
)
print(f"✅ '{COLLECTION_NAME}' 컬렉션 생성 완료")

# =========================================
# 3️⃣ JSON 파일 읽기
# =========================================
with open("fake.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"📂 {len(data)}건 데이터 로드 완료")

# =========================================
# 4️⃣ 임베딩 + Qdrant 업로드
# =========================================
for i, item in enumerate(data, start=1):
    # (1) 임베딩 텍스트 구성
    embed_text = f"{item['공고제목']} - {item['공고내용']}"

    # (2) Ollama에 임베딩 요청
    res = requests.post(OLLAMA_URL, json={
        "model": "mxbai-embed-large",
        "prompt": embed_text
    })

    if res.status_code != 200:
        print(f"❌ 임베딩 실패 (ID={i}): {res.text}")
        continue

    embedding = res.json()["embedding"]

    # (3) Qdrant에 업로드 (벡터 + 메타데이터)
    client.upsert(
        collection_name=COLLECTION_NAME,
        points=[
            models.PointStruct(
                id=i,
                vector=embedding,
                payload={
                    "공고제목": item.get("공고제목"),
                    "공고내용": item.get("공고내용"),
                    "공고일": item.get("공고일"),
                    "공고시작일": item.get("공고시작일"),
                    "공고종료일": item.get("공고종료일"),
                    "공고기간텍스트": item.get("공고기간텍스트"),
                    "니즈": item.get("니즈"),
                    "지역": item.get("지역"),
                    "지원대상": item.get("지원대상"),
                },
            )
        ],
    )

    print(f"✅ [{i}] {item['공고제목']} 저장 완료")

print("\n🎉 모든 데이터 Qdrant 입력 완료!")


/var/folders/sh/7f5kvc6d28599qwlp81kxyc00000gn/T/ipykernel_28332/3865788962.py:19: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


✅ 'policy_notices' 컬렉션 생성 완료
📂 20건 데이터 로드 완료
✅ [1] 2025년도 혁신성장 지원사업 공고 저장 완료
✅ [2] 2025년도 디지털전환 지원사업 공고 저장 완료
✅ [3] 2025년도 중소기업 수출지원 사업 공고 저장 완료
✅ [4] 2025년도 혁신성장 R&D 지원사업 공고 저장 완료
✅ [5] 2025년도 스마트공장 보급사업 공고 저장 완료
✅ [6] 2025년도 글로벌시장 진출지원사업 공고 저장 완료
✅ [7] 2025년도 소상공인 디지털화 지원사업 공고 저장 완료
✅ [8] 2025년도 지역산업 육성사업 공고 저장 완료
✅ [9] 2025년도 AI 기술개발 지원사업 공고 저장 완료
✅ [10] 2025년도 대기업 오픈이노베이션 협력사업 공고 저장 완료
✅ [11] 2025년도 친환경 기술개발 지원사업 공고 저장 완료
✅ [12] 2025년도 지역특화산업 R&D 지원사업 공고 저장 완료
✅ [13] 2025년도 수출바우처 지원사업 공고 저장 완료
✅ [14] 2025년도 스마트서비스 지원사업 공고 저장 완료
✅ [15] 2025년도 창업도약패키지 지원사업 공고 저장 완료
✅ [16] 2025년도 지역청년 창업지원사업 공고 저장 완료
✅ [17] 2025년도 디지털 수출지원사업 공고 저장 완료
✅ [18] 2025년도 중견기업 ESG 혁신지원사업 공고 저장 완료
✅ [19] 2025년도 대기업 상생협력 지원사업 공고 저장 완료
✅ [20] 2025년도 혁신기술 R&D 지원사업 공고 저장 완료

🎉 모든 데이터 Qdrant 입력 완료!


In [3]:
import requests
from qdrant_client import QdrantClient

# =========================================
# 1️⃣ 설정
# =========================================
OLLAMA_URL = "http://localhost:11434/api/embeddings"
QDRANT_URL = "http://localhost:6333"
COLLECTION_NAME = "policy_notices"

# =========================================
# 2️⃣ Qdrant 연결
# =========================================
client = QdrantClient(url=QDRANT_URL)

# =========================================
# 3️⃣ 검색어 입력 (예: 한글 쿼리)
# =========================================
query_text = "AI 기술개발을 지원하는 중소기업 정책"

# Ollama로 쿼리 임베딩 생성
res = requests.post(OLLAMA_URL, json={
    "model": "mxbai-embed-large",
    "prompt": query_text
})
query_vector = res.json()["embedding"]

# =========================================
# 4️⃣ Qdrant에서 벡터 검색 + 메타데이터 출력
# =========================================
results = client.search(
    collection_name=COLLECTION_NAME,
    query_vector=query_vector,
    limit=5  # 상위 5개 결과
)

print(f"\n🔍 검색어: {query_text}")
print("─────────────────────────────")

for i, r in enumerate(results, start=1):
    payload = r.payload
    print(f"[{i}] {payload['공고제목']}")
    print(f"📅 {payload['공고기간텍스트']}")
    print(f"🎯 니즈: {payload['니즈']}, 지역: {payload['지역']}, 지원대상: {payload['지원대상']}")
    print(f"💬 {payload['공고내용'][:100]}...")
    print(f"📈 유사도 점수: {r.score:.4f}")
    print("─────────────────────────────")


🔍 검색어: AI 기술개발을 지원하는 중소기업 정책
─────────────────────────────
[1] 2025년도 AI 기술개발 지원사업 공고
📅 2025.06.25 ~ 2025.08.05
🎯 니즈: ['A', 'F', 'H'], 지역: [20000, 50000], 지원대상: 2
💬 이 사업은 중소기업의 경쟁력 강화를 위해 AI 기술개발을 지원합니다. 신청기관은 산업통상자원부이며 최대 3500만원까지 지원 가능합니다....
📈 유사도 점수: 0.7648
─────────────────────────────
[2] 2025년도 친환경 기술개발 지원사업 공고
📅 2025.07.25 ~ 2025.09.01
🎯 니즈: ['A', 'E'], 지역: [20000, 60000], 지원대상: 2
💬 이 사업은 중소기업의 경쟁력 강화를 위해 친환경 기술개발을 지원합니다. 신청기관은 중소벤처기업부이며 최대 3000만원까지 지원 가능합니다....
📈 유사도 점수: 0.6808
─────────────────────────────
[3] 2025년도 창업도약패키지 지원사업 공고
📅 2025.10.05 ~ 2025.11.10
🎯 니즈: ['F', 'G'], 지역: [10000, 20000], 지원대상: 1
💬 이 사업은 소상공인의 경쟁력 강화를 위해 창업 지원을 제공합니다. 신청기관은 중소벤처기업부이며 최대 2000만원까지 지원 가능합니다....
📈 유사도 점수: 0.6805
─────────────────────────────
[4] 2025년도 중소기업 수출지원 사업 공고
📅 2025.03.05 ~ 2025.04.20
🎯 니즈: ['A', 'G', 'H'], 지역: [10000, 60000], 지원대상: 2
💬 이 사업은 중소기업의 경쟁력 강화를 위해 해외진출을 지원합니다. 신청기관은 산업통상자원부이며 최대 5000만원까지 지원 가능합니다....
📈 유사도 점수: 0.6768
─────────────────────────────
[5] 2025년도 소상공인 디지털화 

/var/folders/sh/7f5kvc6d28599qwlp81kxyc00000gn/T/ipykernel_28332/3181783627.py:31: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


In [4]:
import requests
from qdrant_client import QdrantClient
from qdrant_client.models import Filter, FieldCondition, MatchAny, MatchValue

# =========================================
# 1️⃣ 설정
# =========================================
OLLAMA_URL = "http://localhost:11434/api/embeddings"
QDRANT_URL = "http://localhost:6333"
COLLECTION_NAME = "policy_notices"

# =========================================
# 2️⃣ Qdrant 연결
# =========================================
client = QdrantClient(url=QDRANT_URL)

# =========================================
# 3️⃣ 검색어 입력
# =========================================
query_text = "AI 기술개발 관련 중소기업 지원사업"

# Ollama 임베딩 요청
res = requests.post(OLLAMA_URL, json={
    "model": "mxbai-embed-large",
    "prompt": query_text
})
if res.status_code != 200:
    raise Exception(f"❌ 임베딩 생성 실패: {res.text}")

query_vector = res.json()["embedding"]

# =========================================
# 4️⃣ 필터 조건 설정
# =========================================
# 지역 == 10000 이면서, 니즈 리스트에 'A' 포함
filter_conditions = Filter(
    must=[
        FieldCondition(key="지역", match=MatchAny(any=[10000])),
        FieldCondition(key="니즈", match=MatchValue(value="A")),
    ]
)

# =========================================
# 5️⃣ Qdrant 검색 (벡터 + 필터)
# =========================================
results = client.search(
    collection_name=COLLECTION_NAME,
    query_vector=query_vector,
    limit=5,
    query_filter=filter_conditions
)

# =========================================
# 6️⃣ 결과 출력
# =========================================
print(f"\n🔍 검색어: {query_text}")
print("🧩 필터: 지역=10000, 니즈=A")
print("─────────────────────────────")

if not results:
    print("❗ 조건에 맞는 결과가 없습니다.")
else:
    for i, r in enumerate(results, start=1):
        p = r.payload
        print(f"[{i}] {p['공고제목']}")
        print(f"📅 {p['공고기간텍스트']}")
        print(f"🎯 니즈: {p['니즈']}, 지역: {p['지역']}, 지원대상: {p['지원대상']}")
        print(f"💬 {p['공고내용'][:100]}...")
        print(f"📈 유사도 점수: {r.score:.4f}")
        print("─────────────────────────────")



🔍 검색어: AI 기술개발 관련 중소기업 지원사업
🧩 필터: 지역=10000, 니즈=A
─────────────────────────────
[1] 2025년도 중소기업 수출지원 사업 공고
📅 2025.03.05 ~ 2025.04.20
🎯 니즈: ['A', 'G', 'H'], 지역: [10000, 60000], 지원대상: 2
💬 이 사업은 중소기업의 경쟁력 강화를 위해 해외진출을 지원합니다. 신청기관은 산업통상자원부이며 최대 5000만원까지 지원 가능합니다....
📈 유사도 점수: 0.6618
─────────────────────────────
[2] 2025년도 스마트공장 보급사업 공고
📅 2025.04.05 ~ 2025.05.10
🎯 니즈: ['A', 'D'], 지역: [10000, 40000, 50000], 지원대상: 2
💬 이 사업은 중소기업의 경쟁력 강화를 위해 스마트공장 구축을 지원합니다. 신청기관은 중소벤처기업부이며 최대 2500만원까지 지원 가능합니다....
📈 유사도 점수: 0.6590
─────────────────────────────
[3] 2025년도 혁신성장 지원사업 공고
📅 2025.01.10 ~ 2025.02.20
🎯 니즈: ['A', 'C', 'F'], 지역: [10000, 20000], 지원대상: 2
💬 이 사업은 중소기업의 경쟁력 강화를 위해 스마트공장 구축을 지원합니다. 신청기관은 중소벤처기업부이며 최대 3000만원까지 지원 가능합니다....
📈 유사도 점수: 0.6578
─────────────────────────────
[4] 2025년도 디지털 수출지원사업 공고
📅 2025.11.05 ~ 2025.12.20
🎯 니즈: ['A', 'C', 'D'], 지역: [10000, 30000], 지원대상: 2
💬 이 사업은 중소기업의 경쟁력 강화를 위해 온라인 수출 플랫폼 진출을 지원합니다. 신청기관은 산업통상자원부이며 최대 4000만원까지 지원 가능합니다....
📈 유사도 점수: 0.6575
──────────────────

/var/folders/sh/7f5kvc6d28599qwlp81kxyc00000gn/T/ipykernel_28332/3829364358.py:46: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(
